In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import hashlib
import pandas as pd
import time as t
import hashlib
import json
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
home = 'https://www.hall-tirol.at'
event_site = '/veranstaltungen/'

pages = []
pages.append(home + event_site)
response = requests.get(home + event_site)
soup = BeautifulSoup(response.content, 'html.parser')
events = []

for j in range(len(soup.select('.pageNaviLink'))):
    pages.append(home + soup.select('.pageNaviLink')[j].attrs['href'])


for page in pages:
    t.sleep(3)
    response = requests.get(page)
    soup = BeautifulSoup(response.content, 'html.parser')

    i = 0

    items = soup.select('h3.listEntryTitle')
    for item in items:
        t.sleep(0.1)

        try:
            title = item.select_one('a').string
        except: AttributeError

        try:
            date = soup.select('span.daydate.dayFrom')[i].string
            date = datetime.datetime(int(date[6:]), int(date[3:5]), int(date[0:2]))
            week_day = weekdays[pd.to_datetime(date).weekday()]
        except: AttributeError 

        try:    
            time = soup.select('span.timeFrom')[i].get_text().replace(', ', '')
        except: AttributeError 

        try:
            link = home + str(item.select_one('a').attrs['href'])
        except: AttributeError 

        try:
            location = soup.select('span.listEntryLocation')[i].string
            if location == None or '':
                location = 'Hall in Tirol'
        except: AttributeError

        try:
            short = soup.select('div.listEntryDescription')[i].string.strip().replace('\n', '')
            short = short
        except: AttributeError


        i += 1


        hash = hashlib.sha256()
        hash.update(bytes(title, 'utf-8'))
        hash.update(bytes(location, 'utf-8'))
        hash.update(bytes(str(date), 'utf-8'))
        hash.update(bytes(str(time), 'utf-8'))

        events.append({
            "name": title,
            "date": date,
            "location": location,
            "link": link,
            "short": short,
            "source": "Region Hall in Tirol Homepage",
            "identifier": hash.hexdigest()
        })

In [3]:
weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
home = 'http://www.stromboli.at'
event_site = '/index.php/programm'

pages = []
pages.append(home + event_site)

response = requests.get(home + event_site)
soup = BeautifulSoup(response.content, 'html.parser')
simililarity_dic = []


for tt in range(len(soup.select('.pagination')[0]('a'))):
    try:
        int(soup.select('.pagination')[0]('a')[tt].attrs['title'])
        pages.append(home + soup.select('.pagination')[0]('a')[tt].attrs['href'])
    except: KeyError

for page in pages:
    t.sleep(3)
    response = requests.get(page)
    soup = BeautifulSoup(response.content, 'html.parser')

    i = 0

    items = soup.select('h2.article-title')
    for item in items:
        t.sleep(0.1)

        try:
            title_list = [item.select('a')[0].get_text().strip().split()[2][5:], ' '.join(item.select('a')[0].get_text().strip().split()[3:])]
            title = ' '.join(title_list)
            if title[0] == '-':
                title = title[6:]
            if title == ' ':
                title = soup.select('div.col-md-8')[2*i]('p')[0].get_text().strip().split('\r')[0]
                
        except: AttributeError

        try:
            date = item.select('a')[0].get_text().split()[1]
            day, month, year = int(date[0:2]), int(date[3:5]), int('20'+date[6:8])
            date = datetime.datetime(year, month, day)
            week_day = weekdays[date.weekday()]
        except: AttributeError 

        try:    
            time = item.select('a')[0].get_text().split()[2][0:5]
        except: AttributeError 

        try:
            link = home + item.select('a')[0].attrs['href']
        except: AttributeError 

        try:
            location = 'Stromboli Hall in Tirol'
        except: AttributeError

        try:
            short = soup.select('div.col-md-8')[2*i]('p')[0].get_text().strip()
        except: AttributeError

        i += 1


        hash = hashlib.sha256()
        hash.update(bytes(title, 'utf-8'))
        hash.update(bytes(location, 'utf-8'))
        hash.update(bytes(str(date), 'utf-8'))
        hash.update(bytes(str(time), 'utf-8'))
        
        if type(date) == datetime.datetime:
            events.append({
                "name": title,
                "date": date,
                "location": location,
                "link": link,
                "short": short,
                "source": "Stromboli Hall in Tirol Homepage",
                "identifier": hash.hexdigest()
                    })



In [4]:
final_events = []
duplicates = []
index = []
similarities = []

for i in range(len(events)):
    for j in range(len(events)):
        if events[i]['date'] == events[j]['date']:

            info_1 = events[i]['name'] + ' ' + events[i]['short']
            info_2 = events[j]['name'] + ' ' + events[j]['short']
            information = [info_1, info_2]

            vect = TfidfVectorizer(min_df=1)
            tfidf = vect.fit_transform(information)

            similarity = (tfidf * tfidf.T).A

            if similarity[0][1] > 0.19 and similarity[0][1] < 0.99 and similarity[0][1] not in similarities:
                similarities.append(similarity[0][1])
                index.append(j)
                duplicates.append(events[j])

for i in range(len(events)):
    if i not in index:
        final_events.append(events[i])

In [5]:
len(events), len(final_events), index

(80, 79, [57])

In [6]:
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine

Base = declarative_base()

class Event(Base):
    __tablename__ = 'event'

    id = Column(Integer, primary_key=True)
    identifier = Column(String)
    name = Column(String)
    location = Column(String)
    source = Column(String)
    short = Column(String)
    date = Column(DateTime)
    link = Column(String)



engine = create_engine('sqlite:///events_db_ue6.sqlite')
Base.metadata.create_all(engine)

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# from declarations import Event, Base
# from fhku_event_scraper import get_latest_events

engine = create_engine('sqlite:///events_db_ue6.sqlite')
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()


for event in final_events:
    ev = session.query(Event).filter_by(identifier=event['identifier']).first()
    
    if not ev:
        session.add(Event(
            name=event['name'],
            location=event['location'],
            link=event['link'],
            short=event['short'],
            date=event['date'],
            source=event['source'],
            identifier=event['identifier']
            ))

        

session.commit()
session.close()